In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/moritzm00/utkface-cropped")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vinay7410
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/moritzm00/utkface-cropped


100%|██████████| 116M/116M [00:00<00:00, 853MB/s] 

In [5]:
import tensorflow
from tensorflow import keras
# from tensorflow.keras import layers
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
folder_path = '/content/utkface-cropped/UTKFace'

In [7]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [8]:
len(age)

23708

In [9]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [11]:
df.shape

(23708, 3)

In [12]:
df.head()

,age,gender,img
0,36,1,36_1_0_20170117183326086.jpg.chip.jpg
1,50,0,50_0_0_20170117152322862.jpg.chip.jpg
2,2,0,2_0_4_20161221195155711.jpg.chip.jpg
3,24,1,24_1_2_20170116174412618.jpg.chip.jpg
4,61,0,61_0_3_20170105180813046.jpg.chip.jpg


In [13]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [14]:
train_df.shape

(20000, 3)

In [15]:
test_df.shape

(3708, 3)

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'], # Keep as list of column names
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'], # Keep as list of column names
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [18]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [19]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [20]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [21]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [22]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [27]:
model.fit(train_custom_generator, epochs=10, validation_data=test_custom_generator, steps_per_epoch=len(train_generator), validation_steps=len(test_generator))

TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.